In [ ]:
import csv
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from fnmatch import fnmatch

In [ ]:
filepath = r""

dir_list = os.listdir(filepath)

### Calculate BG-corrected intensity

In [ ]:
root = filepath
pattern_COI = "*COI*.csv"

df_COI = pd.DataFrame()
df_BG = pd.DataFrame()
df_merged = pd.DataFrame()
df_summary = pd.DataFrame()

for path, subdirs, files in os.walk(root):
    
    for name in files:

        
        if fnmatch(name, pattern_COI):

            df_COI_ = pd.read_csv(os.path.join(path, name))
            df_COI_["Frame"] = df_COI_.index
            df_COI_["Name"] = name.rstrip(".csv").lstrip("Values COI ")
            df_COI_["ID"] = df_COI_.X.astype(str) + "__" + df_COI_.Name           
            df_COI = pd.concat([df_COI, df_COI_])

            df_BG_ = pd.read_csv(os.path.join(path, name.replace("COI", "BG")))
            df_BG_["Name"] = name.rstrip(".csv").lstrip("Values COI ")
            df_BG_["ID"] = df_BG_.X.astype(str) + "__" + df_BG_.Name
            df_BG = pd.concat([df_BG, df_BG_])

    
    df_COI = df_COI.reindex()
    df_BG = df_BG.reindex()

    try:
        df_merged = df_COI.merge(df_BG[["Y", "ID"]], on="ID")
        df_merged.drop("ID", axis=1, inplace=True)
        df_merged.rename(columns={"X": "Time", "Y_x": "Intensity_COI", "Y_y": "Intensity_BG"}, inplace=True)
        new_colorder = ["Time", "Intensity_COI", "Intensity_BG", "Name", "Frame"]
        df_merged = df_merged.reindex(columns=new_colorder)
        print(f"{path} processed.")

    except:
        print()
    
    #Insertion of Measurements
    
    if not df_merged.empty:
        df_merged["Intensity_(BGcorrected)"] = df_merged.Intensity_COI - df_merged.Intensity_BG
        df_merged.to_csv(path + "_CorrectedIntensity.csv")
        
        #df_summary["Name"] = path
        df_summary["Mean"] = df_merged.groupby("Frame")["Intensity_(BGcorrected)"].mean()
        df_summary["StDev"] = df_merged.groupby("Frame")["Intensity_(BGcorrected)"].std()
        df_summary["Median"] = df_merged.groupby("Frame")["Intensity_(BGcorrected)"].median()
        df_summary["n"] = df_merged.groupby("Frame")["Intensity_(BGcorrected)"].count()
        df_summary.to_csv(path + "_Summary.csv")
        
    df_COI = pd.DataFrame()
    df_BG = pd.DataFrame()
    df_merged = pd.DataFrame()

print("Done")